# Groningen Mechanical model
This notebook allows running the mechanical model developped at GMG for gas extraction from the Groningen gas field following Smith et al., 2022.

The most complete description of the model can be found in:
Smith, J. D., Heimisson, E. R., Bourne, S. J., & Avouac, J. P. (2022). Stress-based forecasting of induced seismicity with instantaneous earthquake failure functions: applications to the Groningen Gas Reservoir. Earth and Planetary Science Letters, 594, 117697.

A full description of the needed libraries and tested practice on installation and running can be found in the file: Groningen_GMG_Mechanical_model_libraries.txt

A description of the integration of this module with the rest of the modelling workflow can be found in the file:
../README_GroningenModels_GMG.txt

## 1. Libraries and data set-up

### 1.1 Import librairies

In [1]:
print ('Importing librairies...')

# System operations
import sys
from time import time
# Math, Array and Dataformating
import numpy as np
import pandas as pd
import math
from scipy.interpolate import griddata
# Plotting
from shapely.geometry import Point as ShapelyPoint
from shapely.geometry.polygon import Polygon as ShapelyPolygon
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Wedge, Polygon
# Gaussian Smoothing
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

# Gaussian Smoothing Functions
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

from time import time
print ('Librairies imported')

Importing librairies...
Librairies imported


### 1.2 Define the folders to be used

In [2]:
data_folder = '../Reservoir_data/'
results_folder = '../Simulation_results/'

GF_folder = results_folder+'Greens_functions/'
RESdata_folder = data_folder+'RESDictionary_data/'

### 1.3 Load reservoir data

In [3]:
print ('Loading reservoir data...')
# Create the reservoir Geometry
X, Y = np.mgrid[221.250:269.750:0.500, 566.000:615.500:0.500] #dim change
# Load the Thickness data
Thickness = np.load(data_folder + '/ReservoirThickness.npy') / 1000 #dim change
# Load the reservoir outline
Outline = pd.DataFrame(np.load(data_folder + '/ReservoirOutline.npy')) / 1000 #dim change
#Load the compressibility from Smith et al., 2019
CM_dict = np.load(open(data_folder+'Cm.p','rb'),allow_pickle=True)
CM_grd = (CM_dict['InSARGPS'] + CM_dict['RAD'] + CM_dict['OL'])/3 #/1000 / 3600**2 #dim change, Pa-1
print ("Reservoir data loaded")

Loading reservoir data...
Reservoir data loaded


### 1.4 Load pressure data results from module [1]: Reservoir_model

In [4]:
# The list containing pressures, each list element contains the spatial datum at iteration ii, from the mesh loaded right after it
with open(results_folder + 'pressures_list.npy', 'rb') as f:
    pressures_list = np.load(f,allow_pickle=True)
# The mesh from the finite element reservoir simulation, it contains the spatial coordinates of the points where pressures are resolved
with open(results_folder + 'MESH.npy', 'rb') as f:
    MESH = np.load(f,allow_pickle=True)

### 1.5 Import some necessary functions

In [5]:
from util.stress_strain_v2 import create_groningen_object_depth_with_GF, create_groningen_object_depth_no_GF, compute_coulomb_stress_depth
from util.auxiliary import file_exists

## 2. Mechanical model calculations

### 2.1 Compaction and Stress calculations

In [ ]:
# Define the depths at which the calculation will be done 
# z axis is positive toward depth so -10 will calculate at 10m above the reservoir.
depths = [-10]
max_coulomb_stress_depths={}


# Loop for all the depths where we want to calculate the Stresses
for depth in depths:
    # Creation of the Groningen objects with conditional Green's functions for given depths
    # the green's functions will be computed only if they don't exist and will be stored to "Simulation_results/Greens_functions/"
    file_GF_disp = f'G_matrix_disp_{depth}m.pt'
    file_GF_sigma = f'G_matrix_sigma_{depth}m.pt'
    if (file_exists(GF_folder+file_GF_disp) and file_exists(GF_folder+file_GF_sigma)):
        grg = create_groningen_object_depth_no_GF(RESdata_folder,GF_folder,depth)
    else:
        grg = create_groningen_object_depth_with_GF(RESdata_folder,GF_folder,depth)
    
    # Set activate_cuda=True in compute_coulomb_stress if you have a gpu on your computer, much faster than on the cp
    activate_cuda = False
    if activate_cuda:
        #They clean the gpu memory which can be saturated if you run the code several times without restarting the kernel
        from torch.cuda import empty_cache
        empty_cache()
        
    # Calculate the Coulomb stresses in any case and store it to the dictionary max_coulomb_stress_depths.  
    max_coulomb_stress_depths[f'{depth}m'] = compute_coulomb_stress_depth(grg, RESdata_folder, MESH, pressures_list,GF_folder, activate_cuda=False,depth=depth)


Creating Groningen object... 

Observation Points Boundary Check: 100%|███████████████████████████| 6351/6351 [00:03<00:00, 1965.06it/s]


[DONE]
Loading G matrix...
G matrix loaded
Compute displacement and stress on cpu... (could be long, set activate_cuda=True if you have a gpu)


Time-Evolution Values:  60%|█████████████████████████▎                | 603/1000 [09:11<10:32,  1.59s/it]

### 2.2 Subsidence calculation

#### 2.2.1 Creation of the subsidence matrix

In [ ]:
# from subsidence_matrix import create_subsidence_matrix
from util.subsidence_matrix import create_subsidence_matrix_fromGF

#Creates the matrix by which you multiply the pressure to get the subsidence
#This cell can be run once to create the C_matrix in simulation_results/Greens_functions, then you can just load it to avoid recomputing it each time

#Load all the needed data for subsidence matrix
if not file_exists(GF_folder+f'SurfaceGreensFunction_vertical_displacement_only_{1}m.npy'):
    from util.stress_strain_v2 import compute_surfaceGF_z
    compute_surfaceGF_z(RESdata_folder,GF_folder)

    ####Two test lines
depths=[-10]
grg = create_groningen_object_depth_no_GF(RESdata_folder,GF_folder,depths[0])
G_vertical_displacement = np.load(GF_folder+f'SurfaceGreensFunction_vertical_displacement_only_{1}m.npy')
MESH = np.load(results_folder+'MESH.npy')
GF_new_mesh = create_subsidence_matrix_fromGF(MESH, grg.BLOCKS, G_vertical_displacement)
del G_vertical_displacement

#Defining the displacement function
displacement = lambda pressure_array : np.matmul(GF_new_mesh, pressure_array)

#### 2.2.3 Main loop for subsidence calculation

In [ ]:
#Creation of empty list which will contain the deformation results
deformations_list = []

#Always run the previous cell before this one, otherwise pressures_list, extractions_list and deformations_list won't be reset
nb_iterations = len(pressures_list) #60 #One iteration = one month starting Jan/1956

for i in range (1,nb_iterations) :    
    print ('Progression : ' + str(int(i/nb_iterations*100)) + '%', end="\r")
    #Get last values for pressure and gas_extraction
    pres = pressures_list[i]
    #Compute deformations
    new_deformation = displacement(pres)
    deformations_list.append(new_deformation)
    
print ('Deformations computed:')


### 2.3 Save simulation results 

In [ ]:
print ('Saving simulations results...')
with open(results_folder + 'deformations_list.npy', 'wb') as f:
    np.save(f,deformations_list)
with open(results_folder + 'max_coulomb_stresses.npy', 'wb') as f:
    np.save(f,max_coulomb_stress_depths)
print ("[DONE]")

## 3. A few plots

In [ ]:
#Plot 1: plot the simulation results at different times 
iterations = [200,500,700,800]
depth = depths[0] #depth for the coulomb stress plot below the reservoir
max_coulomb_stress = max_coulomb_stress_depths[f'{depths[0]}m']

## import some functions
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve
## import the simulation data
max_coulomb_stress = max_coulomb_stress[:,:,1:]
GasData = np.load(data_folder + 'WellInformation.npy', allow_pickle=True).item() # the information on the well clusters and gas extraction data
wells_locations = np.array(GasData['WellLocations'])[:,1:].astype(np.int32)/1000
wells_names = np.array(GasData['WellLocations'])[:,0]
well_color='olive'
well_scatter_size=50
## Determine the best scales for the plots
### Min and max values for pressure
min_p = np.min(min (pressures_list[iterations[0]:iterations[-1]], key=lambda num_ar: np.min(num_ar)))
max_p = np.max(max (pressures_list[iterations[0]:iterations[-1]], key=lambda num_ar: np.max(num_ar)))
### Min and max values for subsidence
initial_deformations = deformations_list [0]
final_deformations = deformations_list[iterations[-1]]
min_z = min(0, np.min(final_deformations - initial_deformations))
max_z = max(0, np.max(final_deformations - initial_deformations))
### Min and max values for coulomb_stress, kernel definition
kernel = Gaussian2DKernel(6)
min_cs = np.nanmin(max_coulomb_stress)
max_cs = np.nanmax(convolve(max_coulomb_stress[:,:,iterations[-1]],kernel,nan_treatment='fill'))
### Adaptation to km 
RES = grg.RES
x_outline = RES['Outline']['X']/1000
y_outline = RES['Outline']['Y']/1000
res_outline = RES['Outline']/1000
B= grg.BLOCKS
x_reservoir_mesh = (B['X']/1000).to_numpy()
y_reservoir_mesh = (B['Y']/1000).to_numpy()
x_res = RES['X'] / 1000
y_res = RES['Y'] / 1000
x_mesh = MESH[:, 0]
y_mesh = MESH[:, 1]

# Plot the figure
fig,ax = plt.subplots(3,len(iterations),figsize=(4.4*len(iterations),4*(3)+1))
for jj in range(len(iterations)):
    #First subplot : pressure
    ii=0
    quad1 = ax[ii,jj].scatter(x_mesh,y_mesh, c=pressures_list[iterations[jj]],cmap='Blues', vmin = min_p, vmax = max_p, label='Pressure',zorder=1)
    #Second subplot : subsidence
    ii=1
    quad2 = ax[ii,jj].scatter(x_reservoir_mesh,y_reservoir_mesh,50,1e3*(deformations_list[iterations[jj]] - initial_deformations),cmap='Spectral', vmin=1e3*min_z, vmax=1e3*max_z, label='Subsidence', alpha=1,zorder=1)
    #Third subplot : Coulomb_stress
    ii=2
    smoothed_coulomb_stress = convolve(max_coulomb_stress[:,:,iterations[jj]],kernel,nan_treatment='fill')
    quad3 = ax[ii,jj].scatter(x_res.flatten(), y_res.flatten(), c=smoothed_coulomb_stress, cmap='coolwarm', vmin=min_cs, vmax=max_cs,zorder=1)

    for ii in [0,1,2]:
        ax[ii,jj].scatter(wells_locations[:, 0], wells_locations[:, 1], edgecolors=well_color, marker='o', s=well_scatter_size, label='Wells',zorder=2).set_facecolor('None')
        ax[ii,jj].scatter(wells_locations[:, 0], wells_locations[:, 1], edgecolors=well_color, marker='o', s=int(well_scatter_size/10),zorder=2).set_facecolor('None')
        ax[ii,jj].scatter(245, 595, edgecolors='blueviolet', marker='s', s=well_scatter_size*1.3, label='Measurement point',zorder=3).set_facecolor('None')
        ax[ii,jj].scatter(245, 595, edgecolors='blueviolet', marker='s', s=well_scatter_size*0.2, label='Measurement point',zorder=4).set_facecolor('None')
        ax[ii,jj].plot(x_outline,y_outline,'k--',zorder=3)
        ax[ii,jj].text(270, 611, '{}'.format(GasData['Date'][iterations[jj]].strftime('%Y-%m')),horizontalalignment='right',verticalalignment='top',bbox=dict(facecolor='w', edgecolor='k'))
        ax[ii,jj].set_xlim([230.000,270.000])
        ax[ii,jj].set_ylim([565.000,615.000])
        ax[ii,jj].set_xlabel('x (km)')
        ax[ii,jj].set_ylabel('y (km)')
        ax[ii,jj].set_aspect('equal')
plt.tight_layout()

fig.subplots_adjust(left=0.12)
cbar_ax_press = fig.add_axes([0.05, 0.72, 0.02,0.3-0.04])
cbar_ax_disp = fig.add_axes([0.05, 0.4, 0.02,0.3-0.04])
cbar_ax_stress = fig.add_axes([0.05, 0.05,0.02,(0.3-0.04)])
cb_press = fig.colorbar(quad1, cax=cbar_ax_press,orientation='vertical',label='Pressure (MPa)')
cb_press.ax.yaxis.set_ticks_position('left')
cb_press.ax.yaxis.set_label_position('left')
cb_disp = fig.colorbar(quad2, cax=cbar_ax_disp,orientation='vertical',label='Z displacement (mm)')
cb_disp.ax.yaxis.set_ticks_position('left')
cb_disp.ax.yaxis.set_label_position('left')
cb_stress = fig.colorbar(quad3, cax=cbar_ax_stress,orientation='vertical',label='Coulomb stress change (MPa)'.format(depth))
cb_stress.ax.yaxis.set_ticks_position('left')
cb_stress.ax.yaxis.set_label_position('left')
plt.show()


In [ ]:
# Generate the timeseries for the next plot
time = pd.date_range("1956-01-01", periods=nb_iterations-1, freq="M")
pressures = [np.mean(press) for press in pressures_list[1:]]
deformations = [np.mean(deform)-np.mean(deformations_list[0]) for deform in deformations_list]
stresses = [np.mean(convolve(max_coulomb_stress[:,:,ii],kernel,nan_treatment='fill'),axis=(0,1)) for ii in range(max_coulomb_stress.shape[-1])]

In [ ]:
#Plot 2: plot the simulation results timeseries averaged in the reservoir.

fig,ax = plt.subplots(1,figsize=(5*3+1,5))
# Define the spines for different y axes.
twin1 = ax.twinx()
twin2 = ax.twinx()
# Offset the right spine of twin2.  The ticks and label have already been
# placed on the right by twinx above.
twin2.spines.right.set_position(("axes", 1.08))
ax.set_xlabel("Time")
ax.set_ylabel("Average Pressure (MPa)")
twin1.set_ylabel("Average Subsidence (mm)")
twin2.set_ylabel(f"Average MCS calc at {-depths[0]}m above reservoir (MPa)")

#plot the data
#First axis: pressure
# inset_ax # 
p1 = ax.plot(time, pressures,c='navy',label='Average Pressure (MPa)',zorder=1)
#Second axis: subsidence
p2 = twin1.plot(time,1e3*np.array(deformations),c='olive',label='Average Subsidence (mm)', alpha=1,zorder=1)
#Third axis: Coulomb_stress
p3 = twin2.plot(time[:max_coulomb_stress.shape[2]], stresses,label='MCS 5m above reservoir (MPa)', c='darkred',zorder=1)


# correct the spines for different y axes.
ax.yaxis.label.set_color(p1[0].get_color())
twin1.yaxis.label.set_color(p2[0].get_color())
twin2.yaxis.label.set_color(p3[0].get_color())
tkw = dict(size=4, width=1.5)
ax.tick_params(axis='y', colors=p1[0].get_color(), **tkw)
twin1.tick_params(axis='y', colors=p2[0].get_color(), **tkw)
twin2.tick_params(axis='y', colors=p3[0].get_color(), **tkw)
ax.tick_params(axis='x', **tkw)
ax.set_ylim([1,36])

plt.show()
